In [603]:
import pandas as pd
from numpy import nan, mean, median
from statistics import mode
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Home Data pipeline

See [readme](../README.md) 2do section for next steps

In [604]:
# load data

global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data, original_data, original_test_data
def load_data():
    global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data, original_data, original_test_data
    test_data = pd.read_csv("./test.csv")
    original_test_data = test_data.copy()
    original_data = pd.read_csv("./train.csv")
    full_price = original_data.SalePrice
    training___data, validation_data, training___price, validation_price = train_test_split(original_data, full_price)
    training___data = training___data.drop("SalePrice", axis=1)
    validation_data = validation_data.drop("SalePrice", axis=1)
    full_data = original_data.drop("SalePrice", axis=1)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1192,1193,50,RM,60.0,9600,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,5,8,1925,1994,Gambrel,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,TA,TA,Mn,Unf,0,Unf,0,702,702,GasA,Gd,Y,SBrkr,842,630,0,1472,0,0,1,0,3,1,Gd,6,Typ,0,NaN,Detchd,1925.0,Unf,1,250,TA,Fa,P,0,0,84,0,0,0,NaN,GdWo,NaN,0,7,2007,WD,Normal
562,563,30,RL,63.0,13907,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,6,1940,1969,Gable,CompShg,WdShing,Wd Shng,NaN,0.0,TA,TA,CBlock,TA,TA,No,BLQ,290,Unf,0,706,996,GasA,Ex,Y,SBrkr,996,0,0,996,1,0,1,0,3,1,TA,6,Typ,1,Gd,NaN,NaN,NaN,0,0,NaN,NaN,Y,144,0,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
717,718,20,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,5,6,1973,2000,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,CBlock,Gd,TA,No,BLQ,1084,Unf,0,92,1176,GasA,Gd,Y,SBrkr,1178,0,0,1178,0,1,1,1,3,1,Gd,5,Typ,1,Fa,Attchd,1973.0,Unf,2,439,TA,TA,Y,224,0,0,0,0,0,NaN,MnPrv,NaN,0,11,2008,WD,Normal
653,654,50,RM,60.0,10320,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1.5Fin,6,7,1906,1995,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,756,756,GasA,Ex,Y,SBrkr,756,713,0,1469,0,0,1,0,3,1,TA,7,Typ,0,NaN,Detchd,1906.0,Unf,1,216,TA,TA,Y,57,0,239,0,0,0,NaN,MnPrv,NaN,0,6,2008,WD,Normal
255,256,60,RL,66.0,8738,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1999,1999,Gable,CompShg,VinylSd,VinylSd,BrkFace,302.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,975,975,GasA,Ex,Y,SBrkr,1005,1286,0,2291,0,0,2,1,4,1,Gd,8,Typ,1,TA,BuiltIn,1999.0,Fin,2,429,TA,TA,Y,192,0,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,748,70,RM,65.0,11700,Pave,Pave,IR1,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,7,7,1880,2003,Mansard,CompShg,Stucco,Stucco,NaN,0.0,Gd,TA,Stone,TA,Fa,No,Unf,0,Unf,0,1240,1240,GasW,TA,N,SBrkr,1320,1320,0,2640,0,0,1,1,4,1,Gd,8,Typ,1,Gd,Detchd,1950.0,Unf,4,864,TA,TA,N,181,0,386,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal
97,98,20,RL,73.0,10921,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,5,1965,1965,Hip,CompShg,HdBoard,HdBoard,BrkFace,48.0,TA,TA,CBlock,TA,TA,No,Rec,520,Unf,0,440,960,GasA,TA,Y,FuseF,960,0,0,960,1,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1965.0,Fin,1,432,TA,TA,P,120,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
364,365,60,RL,NaN,18800,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,5,1976,1976,Gable,CompShg,HdBoard,HdBoard,BrkFace,120.0,TA,TA,PConc,Gd,TA,Mn,GLQ,712,Unf,0,84,796,GasA,TA,Y,SBrkr,790,784,0,1574,1,0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1976.0,Fin,2,566,TA,TA,Y,306,111,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Normal
234,235,60,RL,NaN,7851,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,625,Unf,0,235,860,GasA,Ex,Y,SBrkr,860,1100,0,1960,1,0,2,1,4,1,Gd,8,Typ,2,TA,BuiltIn,2002.0,Fin,2,44

In [613]:
full_data

,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,Neighborhood,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_20,MSSubClass_85,MSSubClass_90,MSSubClass_30,MSSubClass_160,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_180,MSSubClass_120,MSSubClass_60,MSSubClass_190,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LandContour_Low,LandContour_HLS,LandContour_Lvl,LandContour_Bnk,LotConfig_FR3,LotConfig_Corner,LotConfig_Inside,LotConfig_FR2,LotConfig_CulDSac,Condition1_PosA,Condition1_PosN,Condition1_RRNn,Condition1_RRAn,Condition1_Feedr,Condition1_Artery,Condition1_RRNe,Condition1_RRAe,Condition1_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,Condition2_RRAn,Condition2_Feedr,Condition2_Artery,Condition2_RRAe,Condition2_Norm,BldgType_2fmCon,BldgType_1Fam,BldgType_TwnhsE,BldgType_Twnhs,BldgType_Duplex,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1Story,RoofStyle_Gambrel,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Hip,RoofMatl_CompShg,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_Stone,Exterior1st_BrkComm,Exterior1st_ImStucc,Exterior1st_CBlock,Exterior1st_Wd Sdng,Exterior1st_Stucco,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_AsphShn,Exterior1st_WdShing,Exterior1st_AsbShng,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,masonry other,Foundation_Stone,Foundation_PConc,Foundation_CBlock,Foundation_Slab,Foundation_BrkTil,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,bathz,Heating_GasA,Heating_Wall,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Floor,GarageType_Basment,GarageType_Attchd,GarageType_2Types,GarageType_Detchd,GarageType_BuiltIn,GarageType_CarPort,Fence_nan,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_MnWw,SaleType_WD,SaleType_Con,SaleType_ConLI,SaleType_Oth,SaleType_CWD,SaleType_New,SaleType_ConLD,SaleType_COD,SaleType_ConLw,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Alloca,SaleCondition_AdjLand,Age,IsNew,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,TotalSqft
0,1,65.0,8450,1,0,2,1,7,5,3,2,4,3,1,4,1,4,856,854,0,1710,3,1,3,8,7,0,0,2003.0,2,2,548,3,3,2,0,61,0,0,0,0,3.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,196,0,0,0,0,1,0,0,0,0,706,0,0,0,0,150,3.5,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2566
1,2,80.0,9600,1,0,2,1,6,8,2,2,4,3,4,4,1,4,1262,0,0,1262,3,1,2,6,7,1,3,1976.0,2,2,460,3,3,2,298,0,0,0,0,0,2.666667,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,978,0,0,0,284,2.5,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,31.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2524
2,3,68.0,11250,1,1,2,1,7,5,3,2,4,3,2,4,1,4,920,866,0,1786,3,1,3,6

In [605]:
# define possible column transformations

is_nan = lambda x: x != x
equals = lambda a, b: (is_nan(a) and is_nan(b)) or a == b
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())
all_non_nan_values = lambda column_name: set(training___data[column_name].dropna().unique()) | set(validation_data[column_name].dropna().unique())

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
    return drop(column_name, data_set)

def apply(column_name, data_set, function):
    data_set[column_name] = data_set[column_name].apply(function)
    return data_set

def re_map(column_name, data_set, mapping):
    data_set[column_name] = data_set[column_name].apply(lambda x: mapping[x])
    return data_set

def impute(column_name, data_set, function):
    imputation = function(data_set[column_name])
    data_set[column_name] = data_set[column_name].apply(lambda x: imputation if is_nan(x) else x)
    return data_set

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

def neighborhood_median(column_name, data_set, *_):
    neighborhood_medians = training___data[[column_name, "Neighborhood"]].groupby("Neighborhood").median().to_dict()[column_name]
    fn = lambda x: neighborhood_medians[x["Neighborhood"]] if is_nan(x[column_name]) else x[column_name]
    data_set[column_name] = data_set.apply(fn, axis=1)
    return data_set

def PoolQC(column_name, data_set, *_):
    remap = {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1}
    def fn(x):
        if x[column_name] in remap:
            return remap[x[column_name]]
        if not is_nan(x["PoolArea"]):
            return 1 + (x["OverallQual"]-1)*3/9
        return 0
    data_set[column_name] = data_set.apply(fn, axis=1)
    return data_set

def masonry(column_name, data_set, *_):
    options = dict(
        BrkCmn="Brick Common",
        BrkFace="Brick Face",
        CBlock="Cinder Block",
        Stone="Stone",
    )
    for key in options:
        data_set[f"masonry {options[key]}"] = 0
        for id in range(len(data_set)):
            if data_set[column_name].iloc[id] == key:
                value = data_set["MasVnrArea"].iloc[id]
                # ideally set the above set to the average of that type in that neighborhood (instead of the guess of 75)
                data_set[f"masonry {options[key]}"].iat[id] = value if value > 2 else 75
    
    data_set[f"masonry other"] = 0
    for id in range(len(data_set)):
        if is_nan(data_set["MasVnrType"].iloc[id]) and data_set["MasVnrArea"].iloc[id] > 2:
            data_set["masonry other"].iat[id] = data_set["MasVnrArea"].iloc[id]

    data_set.drop(["MasVnrType", "MasVnrArea"], axis=1, inplace=True)
    return data_set

def fix_basement(data_set):
    basement_fields = [b for b in test_data.keys() if "Bsmt" in b]
    for row_id, row in data_set[basement_fields].iterrows():
        any_nulls = row.isnull().any()
        non_nulls = [type(i) == str for i in row]
        non_zeros = [i > 0 if type(i) in [float, int] else False for key, i in row.items()]
        if any_nulls and (any(non_nulls) or any(non_zeros)):
            for key, i in row.items():
                if is_nan(i):
                    data_set.at[row_id, key] = mode(data_set[key].dropna())

def basement(column_name, data_set, *_):
    fix_basement(data_set)
    options = dict(
        GLQ="Good Living Quarters",
        ALQ="Average Living Quarters",
        BLQ="Below Average Living Quarters",
        Rec="Average Rec Room",
        LwQ="Low Quality",
        Unf="Unfinished",
    )
    for key in options:
        data_set[f"basement {options[key]} sqft"] = 0
        for id in range(len(data_set)):
            if data_set["BsmtFinType1"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF1"].iloc[id]
            if data_set["BsmtFinType2"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF2"].iloc[id]
    
    data_set.drop(["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2"], axis=1, inplace=True)
    return data_set

def basement_unfinished(column_name, data_set, *_):
    unf1 = "basement Unfinished sqft"
    unf2 = "BsmtUnfSF"
    data_set[unf1] = data_set.apply(lambda x: x[unf1] + x[unf2], axis=1)
    
    data_set.drop(["BsmtUnfSF"], axis=1, inplace=True)
    return data_set

def garage_year(column_name, data_set, *_):
    data_set["GarageYrBlt"] = data_set.apply(lambda x: x["YearBuilt"] if is_nan(x["GarageYrBlt"]) else x["GarageYrBlt"], axis=1)
    return data_set

def NA_to_zero(column_name, data_set, *_):
    data_set[column_name] = data_set[column_name].apply(lambda x: 0 if is_nan(x) else x)
    return data_set

def bathz(column_name, data_set, *_):
    cols = ["BsmtFullBath", "FullBath", "BsmtHalfBath", "HalfBath"]
    for col in cols:
        data_set = NA_to_zero(col, data_set)
    data_set[column_name] = data_set["BsmtFullBath"] + data_set["FullBath"] + 0.5*(data_set["BsmtHalfBath"] + data_set["HalfBath"])
    data_set.drop(cols, axis=1, inplace=True)
    return data_set

def is_new(column_name, data_set, *_):
    data_set[column_name] = data_set.apply(lambda x: 1 if x["YrSold"] == x["YearBuilt"] else 0, axis=1)
    return data_set

def age(column_name, data_set, *_):
    data_set[column_name] = data_set.apply(lambda x: x["YrSold"] - x["YearRemodAdd"], axis=1)
    return data_set

def remodeled(column_name, data_set, *_):
    data_set[column_name] = data_set.apply(lambda x: 1 if x["YearBuilt"] != x["YearRemodAdd"] else 0, axis=1)
    return data_set

hoods = original_data[["SalePrice", "Neighborhood"]].groupby("Neighborhood").mean().sort_values("SalePrice").index.tolist()
def hood_check(i):
    if i < 3:
        return 0
    if i >= len(original_data["Neighborhood"].unique()) - 3:
        return 2
    return 1
bougieness = {hood: hood_check(i) for i, hood in enumerate(hoods)}

def total_sqft(column_name, data_set, *_):
    data_set[column_name] = data_set["TotalBsmtSF"] + data_set["GrLivArea"]
    data_set.drop(["TotalBsmtSF"], axis=1, inplace=True)
    return data_set

import inflation
def sale_price(column_name, data_set, *_):
    if column_name in data_set:
        for id in range(len(data_set)):
            data_set[column_name].iat[id] = inflation.adjust(
                price = data_set[column_name].iloc[id],
                from_year = data_set["YrSold"].iloc[id],
                from_month = data_set["MoSold"].iloc[id],
                to_year = 2010,
                to_month = 7,
            )
    data_set.drop(["MoSold", "YrSold"], axis=1, inplace=True)
    return data_set

def post_process(results, data_set, root_data):
    # global original_data
    output = []
    for id, price in enumerate(results):
        id_original = data_set["Id"].iloc[id]
        new_price = inflation.adjust(
            price=price,
            from_year=2010,
            from_month=7,
            to_year=int(root_data["YrSold"][root_data["Id"] == id_original].iloc[0]),
            to_month=int(root_data["MoSold"][root_data["Id"] == id_original].iloc[0]),
        )
        output.append(new_price)
    return results

In [606]:
# specify each column transformation

transformations = [
    ("MSSubClass", (one_hot, all_unique_values("MSSubClass"))),
    ("MSZoning", (impute, mode)),
    ("MSZoning", (MSZoning, None)),
    ("LotFrontage", (neighborhood_median, None)),
    ("Street", (re_map, {"Pave": 1, "Grvl": 0})),
    ("Alley", (drop, None)),
    ("LotShape", (re_map, {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3})),
    # "LandContour": (re_map, {"Lvl": 1, "Bnk": 3, "HLS": 2, "Low": 0}),
    ("LandContour", (one_hot, all_unique_values("LandContour"))),
    ("Utilities", (drop, None)),
    ("LotConfig", (one_hot, all_unique_values("LotConfig"))),
    ("LandSlope", (re_map, {"Gtl": 2, "Mod": 1, "Sev": 0})),
    ("Neighborhood", (re_map, bougieness)),
    ("Condition1", (one_hot, all_unique_values("Condition1"))),
    ("Condition2", (one_hot, all_unique_values("Condition2"))),
    ("BldgType", (one_hot, all_unique_values("BldgType"))),
    ("HouseStyle", (one_hot, all_unique_values("HouseStyle"))),
    ("RoofStyle", (one_hot, all_unique_values("RoofStyle"))),
    ("RoofMatl", (one_hot, all_unique_values("RoofMatl"))),
    ("Exterior1st", (impute, mode)),
    ("Exterior1st", (one_hot, all_unique_values("Exterior1st"))),
    ("Exterior2nd", (drop, None)),
    ("MasVnrType", (masonry, None)),
    ("ExterQual", (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0})),
    ("ExterCond", (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0})),
    ("Foundation", (one_hot, all_unique_values("Foundation"))),
    ("TotalBsmtSF", (NA_to_zero, None)),
    ("BsmtUnfSF", (NA_to_zero, None)),
    ("TotalBsmtSF", (basement, None)),
    ("BsmtQual", (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0, nan: 0})),
    ("BsmtCond", (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0, nan: 0})),
    ("BsmtExposure", (re_map, {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0, nan: 0})),
    ("BsmtUnfSF", (basement_unfinished, None)),
    ("bathz", (bathz, None)),
    ("Heating", (one_hot, all_unique_values("Heating"))),
    ("HeatingQC", (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0})),
    ("CentralAir", (re_map, {"Y": 1, "N": 0})),
    ("Electrical", (re_map, {"SBrkr": 4, "FuseA": 3, "FuseF": 2, "FuseP": 1, "Mix": 0, nan: 0})),
    ("KitchenQual", (impute, mode)),
    ("KitchenQual", (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0})),
    ("Functional", (impute, mode)),
    ("Functional", (re_map, {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0})),
    ("FireplaceQu", (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 0})),
    ("GarageType", (one_hot, all_non_nan_values("GarageType"))),
    ("GarageYrBlt", (garage_year, None)),
    ("GarageFinish", (re_map, {"Fin": 3, "RFn": 2, "Unf": 1, "NA": 0, nan: 0})),
    ("GarageCars", (NA_to_zero, None)),
    ("GarageQual", (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0, nan: 0})),
    ("GarageCond", (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0, nan: 0})),
    ("PavedDrive", (re_map, {"Y": 2, "P": 1, "N": 0})),
    ("PoolQC", (PoolQC, None)),
    ("Fence", (one_hot, all_unique_values("Fence"))),
    ("MiscFeature", (drop, None)),
    ("SaleType", (impute, mode)),
    ("SaleType", (one_hot, all_unique_values("SaleType"))),
    ("SaleCondition", (one_hot, all_unique_values("SaleCondition"))),
    ("Age", (age, None)),
    ("IsNew", (is_new, None)),
    ("YrSold", (one_hot, all_unique_values("YrSold"))),
    ("MoSold", (one_hot, all_unique_values("MoSold"))),
    ("YearBuilt", (drop, None)),
    ("YearRemodAdd", (drop, None)),
    ("TotalSqft", (total_sqft, None)),
]

In [607]:
def manual_transformations():
    for col in ["GarageFinish", "GarageQual", "GarageCond"]:
        test_data[col].iat[666] = mode(test_data[col])
    test_data["GarageType"].iat[1116] = nan
    test_data["GarageArea"].iat[1116] = 0

In [608]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations:
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming MSZoning
transforming LotFrontage
transforming Street
transforming Alley
transforming LotShape
transforming LandContour
transforming Utilities
transforming LotConfig
transforming LandSlope
transforming Neighborhood
transforming Condition1
transforming Condition2
transforming BldgType
transforming HouseStyle
transforming RoofStyle
transforming RoofMatl
transforming Exterior1st
transforming Exterior1st
transforming Exterior2nd
transforming MasVnrType
transforming ExterQual
transforming ExterCond
transforming Foundation
transforming TotalBsmtSF
transforming BsmtUnfSF
transforming TotalBsmtSF
transforming BsmtQual
transforming BsmtCond
transforming BsmtExposure
transforming BsmtUnfSF
transforming bathz
transforming Heating
transforming HeatingQC
transforming CentralAir
transforming Electrical
transforming KitchenQual
transforming KitchenQual
transforming Functional
transforming Functional
transforming FireplaceQu
transforming Garag

,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,Neighborhood,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_20,MSSubClass_85,MSSubClass_90,MSSubClass_30,MSSubClass_160,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_180,MSSubClass_120,MSSubClass_60,MSSubClass_190,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LandContour_Low,LandContour_HLS,LandContour_Lvl,LandContour_Bnk,LotConfig_FR3,LotConfig_Corner,LotConfig_Inside,LotConfig_FR2,LotConfig_CulDSac,Condition1_PosA,Condition1_PosN,Condition1_RRNn,Condition1_RRAn,Condition1_Feedr,Condition1_Artery,Condition1_RRNe,Condition1_RRAe,Condition1_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,Condition2_RRAn,Condition2_Feedr,Condition2_Artery,Condition2_RRAe,Condition2_Norm,BldgType_2fmCon,BldgType_1Fam,BldgType_TwnhsE,BldgType_Twnhs,BldgType_Duplex,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1Story,RoofStyle_Gambrel,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Hip,RoofMatl_CompShg,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_Stone,Exterior1st_BrkComm,Exterior1st_ImStucc,Exterior1st_CBlock,Exterior1st_Wd Sdng,Exterior1st_Stucco,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_AsphShn,Exterior1st_WdShing,Exterior1st_AsbShng,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,masonry other,Foundation_Stone,Foundation_PConc,Foundation_CBlock,Foundation_Slab,Foundation_BrkTil,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,bathz,Heating_GasA,Heating_Wall,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Floor,GarageType_Basment,GarageType_Attchd,GarageType_2Types,GarageType_Detchd,GarageType_BuiltIn,GarageType_CarPort,Fence_nan,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_MnWw,SaleType_WD,SaleType_Con,SaleType_ConLI,SaleType_Oth,SaleType_CWD,SaleType_New,SaleType_ConLD,SaleType_COD,SaleType_ConLw,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Alloca,SaleCondition_AdjLand,Age,IsNew,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,TotalSqft
945,946,98.0,8820,1,0,2,1,5,6,2,2,3,3,1,2,1,4,1188,561,120,1869,2,1,2,7,7,0,0,1963.0,1,2,456,3,3,2,48,0,244,0,0,0,2.333333,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1088,0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,13.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2957
378,379,88.0,11394,1,0,2,2,9,2,3,2,5,3,3,4,1,4,1856,0,0,1856,1,1,4,8,7,1,5,2010.0,3,3,834,3,3,2,113,0,0,0,0,0,3.666667,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,350,0,0,1,0,0,0,0,1445,0,0,0,0,411,2.5,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,3712
126,127,24.0,4928,1,1,2,1,6,5,2,2,4,3,1,2,1,4,958,0

In [609]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,Neighborhood,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_20,MSSubClass_85,MSSubClass_90,MSSubClass_30,MSSubClass_160,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_180,MSSubClass_120,MSSubClass_60,MSSubClass_190,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LandContour_Low,LandContour_HLS,LandContour_Lvl,LandContour_Bnk,LotConfig_FR3,LotConfig_Corner,LotConfig_Inside,LotConfig_FR2,LotConfig_CulDSac,Condition1_PosA,Condition1_PosN,Condition1_RRNn,Condition1_RRAn,Condition1_Feedr,Condition1_Artery,Condition1_RRNe,Condition1_RRAe,Condition1_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,Condition2_RRAn,Condition2_Feedr,Condition2_Artery,Condition2_RRAe,Condition2_Norm,BldgType_2fmCon,BldgType_1Fam,BldgType_TwnhsE,BldgType_Twnhs,BldgType_Duplex,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1Story,RoofStyle_Gambrel,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Hip,RoofMatl_CompShg,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_Stone,Exterior1st_BrkComm,Exterior1st_ImStucc,Exterior1st_CBlock,Exterior1st_Wd Sdng,Exterior1st_Stucco,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_AsphShn,Exterior1st_WdShing,Exterior1st_AsbShng,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,masonry other,Foundation_Stone,Foundation_PConc,Foundation_CBlock,Foundation_Slab,Foundation_BrkTil,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,bathz,Heating_GasA,Heating_Wall,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Floor,GarageType_Basment,GarageType_Attchd,GarageType_2Types,GarageType_Detchd,GarageType_BuiltIn,GarageType_CarPort,Fence_nan,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_MnWw,SaleType_WD,SaleType_Con,SaleType_ConLI,SaleType_Oth,SaleType_CWD,SaleType_New,SaleType_ConLD,SaleType_COD,SaleType_ConLw,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Alloca,SaleCondition_AdjLand,Age,IsNew,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,TotalSqft
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.00000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.0,365.0,365.0,365.0,365.000000,365.0,365.000000,365.000000,36

# Fit, cross-validate, & compare

In [610]:
from xgboost import XGBRegressor
from sigfig import round
pretty = lambda x: round(str(x), decimals=2, spacer=",", spacing=3)
from sklearn.metrics import mean_absolute_error
from capture_output import Capturing
from random import shuffle

load_data()
for column_name, (transform, option) in transformations:
    full_data = transform(column_name, full_data, option)

outlier_ids = [523, 1298]
full_data = full_data.drop(outlier_ids).reset_index(drop=True)
full_price = full_price.drop(outlier_ids).reset_index(drop=True)
print(f"Dropped {len(original_data) - len(full_data)} rows")

def generate_cross_validation_sets(full_data, sets=5):
    indicies = [i for i in range(len(full_data))]
    shuffle(indicies)
    set_size = len(full_data) // sets
    for n in range(sets):
        yield indicies[n*set_size:(n+1)*set_size]

errors = []
iterations = []
for indicies_subset in generate_cross_validation_sets(full_data, sets=5):
    training___data = full_data.drop(indicies_subset)
    training___price = full_price.drop(indicies_subset)
    validation_data = full_data.iloc[indicies_subset]
    validation_price = full_price.iloc[indicies_subset]

    model = None
    model = XGBRegressor(n_estimators=1_000, learning_rate=0.05, early_stopping_rounds=5)
    with Capturing() as output:
        model.fit(training___data.drop("Id", axis=1), training___price, eval_set=[(validation_data.drop("Id", axis=1), validation_price)])

    predictions = model.predict(validation_data.drop("Id", axis=1))

    error = float(mean_absolute_error(validation_price, predictions))
    print(f"Error: {pretty(error)}", f"Iterations: {len(output)}")
    errors.append(error)
    iterations.append(len(output))

print(f"Mean Absolute Error: {pretty(mean(errors))}")
print(f"Optimal # Iterations(?): {mean(iterations)}")

Dropped 2 rows
Error: 15,715.86 Iterations: 212
Error: 14,572.89 Iterations: 184
Error: 15,086.62 Iterations: 196
Error: 14,865.62 Iterations: 180
Error: 16,833.75 Iterations: 166
Mean Absolute Error: 15,414.95
Optimal # Iterations(?): 187.6


## Saved results (with Random Forest model)

- Normie: Mean Absolute Error: 17787.279715068493: 16742.85882191781...19406.53876712329
- No Alley: Mean Absolute Error of 17652.627992694062: 16217.109150684932...19793.63994520548
- Basic zoning: Mean Absolute Error of 18059.097720547943: 16746.11709589041...19786.975287671234

# Generate Output

In [611]:
from xgboost import XGBRegressor

load_data()
manual_transformations()
for column_name, (transform, option) in transformations:
    full_data = transform(column_name, full_data, option)
    test_data = transform(column_name, test_data, option)

model = None
model = XGBRegressor(n_estimators=1_000, learning_rate=0.05)
model.fit(full_data.drop("Id", axis=1), full_price)

predictions = model.predict(test_data.drop("Id", axis=1))

submission = pd.DataFrame({'Id': test_data.index + 1461, 'SalePrice': predictions})
submission.to_csv('submission.csv', index=False)
submission

,Id,SalePrice
0,1461,111641.601562
1,1462,159295.171875
2,1463,186162.546875
3,1464,188304.109375
4,1465,191325.828125
...,...,...
1454,2915,77416.710938
1455,2916,79097.078125
1456,2917,150133.437500
1457,2918,119475.484375


In [612]:
# manual validation
from itertools import chain
# load_data()
m_fields = [g for g in test_data.keys() if "masonry" in g] + ["Id"]
# for id, row in chain(test_data[garage_fields].iterrows(), full_data[garage_fields].iterrows()):
#     if row.isnull().any():
#         if any([type(i) == str for i in row]) or any([i > 0 if type(i) in [float, int] and key != "Id" else False for key, i in row.items()]):
#             print(row)
Idz = [1670, 2320, 2453, 2611, 625, 689, 774, 1231, 1242, 1301, 1335]
output = pd.DataFrame(columns=full_data.keys())
for s3t in test_data, full_data:
    output = pd.concat([output, s3t[s3t["Id"].isin(Idz)]])
output[m_fields]
# full_data.drop("Id", axis=1).isnull().values.any()
output = pd.DataFrame(columns=full_data.keys())
for id, row in test_data.iterrows():
    if row.isnull().any():
        output = pd.concat([output, test_data[test_data["Id"] == row["Id"]]])
# test_data.isnull().any(axis=1).sum()
output

/tmp/ipykernel_1540/1064105884.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = pd.concat([output, s3t[s3t["Id"].isin(Idz)]])


,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,Neighborhood,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_20,MSSubClass_85,MSSubClass_90,MSSubClass_30,MSSubClass_160,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_180,MSSubClass_120,MSSubClass_60,MSSubClass_190,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LandContour_Low,LandContour_HLS,LandContour_Lvl,LandContour_Bnk,LotConfig_FR3,LotConfig_Corner,LotConfig_Inside,LotConfig_FR2,LotConfig_CulDSac,Condition1_PosA,Condition1_PosN,Condition1_RRNn,Condition1_RRAn,Condition1_Feedr,Condition1_Artery,Condition1_RRNe,Condition1_RRAe,Condition1_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,Condition2_RRAn,Condition2_Feedr,Condition2_Artery,Condition2_RRAe,Condition2_Norm,BldgType_2fmCon,BldgType_1Fam,BldgType_TwnhsE,BldgType_Twnhs,BldgType_Duplex,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1Story,RoofStyle_Gambrel,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Hip,RoofMatl_CompShg,RoofMatl_WdShake,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_Stone,Exterior1st_BrkComm,Exterior1st_ImStucc,Exterior1st_CBlock,Exterior1st_Wd Sdng,Exterior1st_Stucco,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_AsphShn,Exterior1st_WdShing,Exterior1st_AsbShng,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,masonry other,Foundation_Stone,Foundation_PConc,Foundation_CBlock,Foundation_Slab,Foundation_BrkTil,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,bathz,Heating_GasA,Heating_Wall,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Floor,GarageType_Basment,GarageType_Attchd,GarageType_2Types,GarageType_Detchd,GarageType_BuiltIn,GarageType_CarPort,Fence_nan,Fence_MnPrv,Fence_GdWo,Fence_GdPrv,Fence_MnWw,SaleType_WD,SaleType_Con,SaleType_ConLI,SaleType_Oth,SaleType_CWD,SaleType_New,SaleType_ConLD,SaleType_COD,SaleType_ConLw,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Family,SaleCondition_Alloca,SaleCondition_AdjLand,Age,IsNew,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,TotalSqft
